# Gallery Example: M/Erl/1 Tandem Network

This example demonstrates a simple 2-queue tandem network:
- **Arrivals**: Exponential (Poisson process)
- **Service**: Erlang at both queues
- **Servers**: 1 server per queue
- **Scheduling**: FCFS

This is a special case of the linear network with n=2.

In [ ]:
from line_solver import *
GlobalConstants.set_verbose(VerboseLevel.STD)

In [ ]:
def gallery_merl1_tandem():
    """Create M/Erl/1-Tandem queueing network (2 queues in series)"""
    model = Network('M/Erl/1-Tandem')
    
    # Block 1: nodes
    source = Source(model, 'mySource')
    queue1 = Queue(model, 'Queue1', SchedStrategy.FCFS)
    queue2 = Queue(model, 'Queue2', SchedStrategy.FCFS)
    sink = Sink(model, 'mySink')
    
    # Block 2: classes
    oclass = OpenClass(model, 'myClass')
    
    # Exponential arrivals
    source.set_arrival(oclass, Exp(1))
    
    # Erlang service with increasing order
    queue1.set_service(oclass, Erlang.fit_mean_and_order(0.1, 1))
    queue2.set_service(oclass, Erlang.fit_mean_and_order(0.1, 2))
    
    # Block 3: topology
    model.link(Network.serial_routing(source, queue1, queue2, sink))
    
    return model

# Create the model
model = gallery_merl1_tandem()

## About M/Erl/1 Tandem

This 2-queue tandem model has:
- **Queue 1**: Erlang(1) service = Exponential (SCV = 1.0)
- **Queue 2**: Erlang(2) service (SCV = 0.5)

The decreasing variability at the second queue means:
- Output of Queue 1 is more regular than its input
- Queue 2 experiences less variability
- This can lead to overall better performance than having high variability at both stations

In [ ]:
# Solve with multiple solvers
print("\n=== Solver Results ===")

# CTMC Solver
solver_ctmc = CTMC(model, cutoff=15)
avg_table_ctmc = solver_ctmc.avg_table()
print("\nCTMC Solver:")
print(avg_table_ctmc)

# Fluid Solver
solver_fluid = FLD(model)
avg_table_fluid = solver_fluid.avg_table()
print("\nFluid Solver:")
print(avg_table_fluid)